In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()

In [47]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'heart-disease'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       target_recall=0.947,
                       binary_classifier_model_selection_criteria='precision_at_target_recall',
                       epochs=15)

In [48]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # convert the df into a matrix for ease of splitting
    df_matrix = transaction_df.values
    
    # shuffle the data
    np.random.seed(seed)
    np.random.shuffle(df_matrix)
    
    # split the data
    train_size = int(df_matrix.shape[0] * train_frac)
    # features are all but last column
    train_features  = df_matrix[:train_size, :-1]
    # class labels *are* last column
    train_labels = df_matrix[:train_size, -1]
    # test data
    test_features = df_matrix[train_size:, :-1]
    test_labels = df_matrix[train_size:, -1]
    
    return (train_features, train_labels), (test_features, test_labels)

In [49]:
df = pd.read_csv("data/heart.csv") # 'DataFrame' shortened to 'df'
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(df, train_frac=0.8)

In [50]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [51]:
# convert features/labels to numpy
train_x_np = train_features.astype('float32')
train_y_np = train_labels.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

In [52]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

2020-05-04 16:31:17 Starting - Starting the training job...
2020-05-04 16:31:19 Starting - Launching requested ML instances...
2020-05-04 16:32:16 Starting - Preparing the instances for training......
2020-05-04 16:33:15 Downloading - Downloading input data...
2020-05-04 16:33:47 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[05/04/2020 16:33:49 INFO 140462077089600] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_met

# Deploy

In [60]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Using already existing model: linear-learner-2020-05-04-16-31-17-719


---------------!CPU times: user 256 ms, sys: 8.26 ms, total: 264 ms
Wall time: 7min 32s


## Evaluating model

In [61]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # find predictions based on features
    test_preds = np.concatenate([np.array([prediction.label['predicted_label'].float32_tensor.values[0] for prediction in predictor.predict(test_features)]) 
                                 ])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = 2*((precision*recall)/(precision+recall))
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))

        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}

In [62]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features[:20].astype('float32'), 
                   test_labels[:20], 
                   verbose=True) # verbose means we'll print out the metrics

Metrics for simple, LinearLearner.

prediction (col)  0.0  1.0
actual (row)              
0.0                 6    1
1.0                 0   13

Recall:     1.000
Precision:  0.929
Accuracy:   0.950
F1:         0.963



## Testing for model robustness

In [141]:
def getAvgMetrics(list,skip):
    avg_score={'TP': 0,
                 'FP': 0,
                 'FN': 0,
                 'TN': 0,
                 'Precision': 0,
                 'Recall': 0,
                 'Accuracy': 0}
    i=len(list)
    for x in list:
        metrics=evaluate(linear_predictor, 
                   test_features[x-skip:x].astype('float32'), 
                   test_labels[x-skip:x], 
                   verbose=False
                   )
        avg_score={'TP': (avg_score['TP']+metrics['TP']), 'FP': (avg_score['FP']+metrics['FP']), 'FN': (avg_score['FN']+metrics['FN']), 'TN': (avg_score['TN']+metrics['TN']), 
                'Precision': (avg_score['Precision']+metrics['Precision']), 'Recall': (avg_score['Recall']+metrics['Recall']), 'Accuracy': (avg_score['Accuracy']+metrics['Accuracy'])}
    return {'TP': (avg_score['TP'])/i, 'FP': (avg_score['FP'])/i, 'FN': (avg_score['FN'])/i, 'TN': (avg_score['TN'])/i, 
                'Precision': (avg_score['Precision'])/i, 'Recall': (avg_score['Recall'])/i, 'Accuracy': (avg_score['Accuracy'])/i}

In [151]:
at_20=[
    20,30,40,
    50,60]

getAvgMetrics(at_20,20)

{'TP': 12.2,
 'FP': 3.0,
 'FN': 0.0,
 'TN': 4.8,
 'Precision': 0.8019423558897243,
 'Recall': 1.0,
 'Accuracy': 0.85}

In [143]:
at_30=[30,40,50,60]
getAvgMetrics(at_30,30)

{'TP': 19.0,
 'FP': 4.75,
 'FN': 0.0,
 'TN': 6.25,
 'Precision': 0.7977380952380952,
 'Recall': 1.0,
 'Accuracy': 0.8416666666666666}

In [145]:
at_40=[40,50,60]
getAvgMetrics(at_40,40)

{'TP': 25.333333333333332,
 'FP': 6.333333333333333,
 'FN': 0.0,
 'TN': 8.333333333333334,
 'Precision': 0.7989573150863473,
 'Recall': 1.0,
 'Accuracy': 0.8416666666666667}

In [146]:
at_50=[50,60]
getAvgMetrics(at_50,50)

{'TP': 30.5,
 'FP': 7.5,
 'FN': 0.0,
 'TN': 12.0,
 'Precision': 0.8021483021483021,
 'Recall': 1.0,
 'Accuracy': 0.85}

In [147]:
evaluate(linear_predictor, 
               test_features.astype('float32'), 
               test_labels, 
               verbose=True)

prediction (col)  0.0  1.0
actual (row)              
0.0                15    8
1.0                 0   38

Recall:     1.000
Precision:  0.826
Accuracy:   0.869
F1:         0.905



{'TP': 38,
 'FP': 8,
 'FN': 0,
 'TN': 15,
 'Precision': 0.8260869565217391,
 'Recall': 1.0,
 'Accuracy': 0.8688524590163934}

In [ ]:
# Delete Endpoint
boto3.client('sagemaker').delete_endpoint(EndpointName=linear_predictor.endpoint)